# PC lab: Generative models

In this practical, we’ll introduce a class of unsupervised deep learning models, called generative models, that enables us to learn the underlying, unknown distribution of a given dataset. Using these models, we can generate new samples that resemble the observed training data. 

We’ll begin by constructing an autoencoder network — a neural network architecture designed to encode and reconstruct data, learning efficient representations. Finally, we’ll extend our autoencoder into a generative model, known as a variational autoencoder (VAE), capable of producing realistic new samples. For demonstration, we will train and test the VAE on the MNIST dataset. 

## 1.1 Creating an autoencoder

Typically, an autoencoder will look something like this:

<img src='https://miro.medium.com/max/1400/1*gzJAJDLDavH_W7Zv2M2J7w.png' width = 600>

In essence, the first part of the network (the encoder) compresses the input to a lower dimensional space. It does this by (1) first using convolutions, (2) flattening, and (3) a linear layer at the end. The second part (the decoder) undergoes the same steps, but in reverse.

In order to make an autoencoder, we can copy the model from the CNN PC lab, use it as an encoder, and then make the inverse of that model as the decoder, and place the two in sequence.

As a reminder, this was the CNN Classifier from the previous PC lab, with some small changes:

In [ ]:
import torch
import torch.nn as nn

class ResidualBlock(nn.Module):
    def __init__(self, hidden_dim = 64, kernel_size = 5):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(hidden_dim, hidden_dim, kernel_size, padding = "same"),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.BatchNorm2d(hidden_dim)
        )
    def forward(self, x):
        return self.net(x) + x

class Encoder(nn.Module):
    def __init__(self, bottleneck_size = 8):
        super().__init__()
        self.cnn = nn.Sequential(
            nn.Conv2d(1, 8, 5),
            ResidualBlock(8),
            nn.MaxPool2d(2),
            ResidualBlock(8),
            nn.Conv2d(8, 16, 5),
            ResidualBlock(16),
            nn.MaxPool2d(2),
            ResidualBlock(16),
            nn.Conv2d(16, 16, 3),
        )
        self.flatten = nn.Flatten()
        self.MLP = nn.Linear(16 * 2 * 2, bottleneck_size)

    def forward(self, x):
        return self.MLP(self.flatten(self.cnn(x)))


For the decoder, we need inverse convolutions (and maybe inverse max pooling) operations to upscale our image again from its compressed representation to its original dimensions.

In PyTorch, we have these options:

- [ConvTranspose2d](https://pytorch.org/docs/stable/generated/torch.nn.ConvTranspose2d.html) also visualized [here](https://pytorch.org/docs/stable/generated/torch.nn.ConvTranspose2d.html), the inverse of convolutions, also called deconvolution.
- [Upsample](https://pytorch.org/docs/stable/generated/torch.nn.Upsample.html#torch.nn.Upsample), performing simple non-parametric upscaling of inputs with any of a number of methods such as bilinear or bicubic upscaling.
- [MaxUnpool2d](https://pytorch.org/docs/stable/generated/torch.nn.MaxUnpool2d.html?highlight=maxunpool#torch.nn.MaxUnpool2d), performing the inverse of a previous max pooling operation. Essentially putting the maximal elements back in the location/index where they originally appeared before the previous max pooling. As such, this operation can only be used conjoined with a paired max pooling operation. (see examples in documentation).

For this PC lab, we simply recommend to replace Convolutions with `ConvTranspose2d` and Max-Pooling layers with `Upsample`.

Also, you will need to [Unflatten](https://pytorch.org/docs/stable/generated/torch.nn.Unflatten.html) your image after flattening it:

In [ ]:
# Illustration of unflatten:
# The dimensions we want:
C = 64
H = 16
W = 16

unflatten = nn.Unflatten(1, (C, H, W))
x = torch.randn(8, C*H*W) # imagine you have a tensor of this size before unflattening
print(x.shape)
y = unflatten(x)
print(y.shape)

The following code illustrates the use of [Upsample](https://pytorch.org/docs/stable/generated/torch.nn.Upsample.html#torch.nn.Upsample):

In [ ]:
upsample = nn.Upsample(scale_factor = 2)
x = torch.randn(8, 64, 16, 16)
print(x.shape)
y = upsample(x)
print(y.shape)

<div class="alert alert-success">
<b>EXERCISE:</b>
<p> Implement a Decoder by creating the inverse of the Encoder model in the cell above. Create an autoencoder model that combines the Encoder and decoder.</p>
</div>

In [ ]:
class Decoder(nn.Module):
    def __init__(self, bottleneck_size = 8):
        super().__init__()
        self.MLP = nn.Linear(...) # Your code here
        self.flatten = nn.Unflatten(...) # Your code here
        self.cnn = nn.Sequential(
            ... # your code here, try to invert the CNN of the encoder
        )

    def forward(self, x):
        return self.cnn(self.flatten(self.MLP(x)))


class AutoEncoder(nn.Module):
    def __init__(self, bottleneck_size = 8):
        super().__init__()
        self.encoder = Encoder(bottleneck_size = bottleneck_size)
        self.decoder = Decoder(bottleneck_size = bottleneck_size)

    def encode(self, x):
        return self.encoder(x)
    def decode(self, x):
        return self.decoder(x)
    def forward(self, x):
        z = self.encode(x)
        return self.decode(z)

model = AutoEncoder(bottleneck_size = 8)

In [ ]:
x = torch.randn(8, 1, 28, 28)
print("original shape of x:")
print(x.shape)
print("Testing encoder:")
x_encoded = model.encode(x)
print(x_encoded.shape)
print("Testing decoder:")
x_decoded = model.decode(x_encoded)
print(x_decoded.shape)
print("Testing whole model at once:")
y = model(x)
print(y.shape)

After successfully implementing the autoencoder, we can work on some real data. The following code loads in the MNIST data. It's exactly copy-pasted from the CNN PC-lab so you can skip it.

In [ ]:
from torchvision import datasets
from torchvision.transforms import ToTensor
import numpy as np

train_data = datasets.MNIST(
    root = 'data',
    train = True,
    transform = ToTensor(),
    download = True,
)
test_data = datasets.MNIST(
    root = 'data',
    train = False,
    transform = ToTensor()
)

X_train = train_data.data
y_train = train_data.targets

X_test = test_data.data
y_test = test_data.targets

X_train = X_train / 255
X_test = X_test / 255

X_train = X_train.unsqueeze(1)
X_test = X_test.unsqueeze(1)

np.random.seed(42)
train_indices, val_indices = np.split(np.random.permutation(len(X_train)), [int(len(X_train)*0.8)])
X_val = X_train[val_indices]
y_val = y_train[val_indices]
X_train = X_train[train_indices]
y_train = y_train[train_indices]

train_dataset = torch.utils.data.TensorDataset(X_train, y_train)
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=16, pin_memory=True, shuffle=True)

val_dataset = torch.utils.data.TensorDataset(X_val, y_val)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=16, pin_memory=True, shuffle=True)

test_dataset = torch.utils.data.TensorDataset(X_test, y_test)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=16, pin_memory=True, shuffle=True)

The following code trains the resulting autoencoder model on MNIST, using similar code as in previous PC labs. Some notes:
- We are reconstructing pixels. These are real-valued. Hence, this is a regression task and we're using the Mean squared error loss.
- Our batches return an X and a Y, but we're not using the labels Y here. Instead, the true "labels" for our loss is simply X.
- In contrast to previous PC labs - this code uses the GPU (note the `.to("cuda")` calls). This considerably speeds up training. If you're running this locally and don't have a GPU, delete these calls. If you're running on colab but it still returns an error: Go to `Runtime > Change runtime type`, and select an instance with GPUs.

In [ ]:
N_EPOCHS = 20

model = AutoEncoder(bottleneck_size = 8).to("cuda")

loss_function = nn.MSELoss()

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

for i in range(1, N_EPOCHS + 1):
    all_losses = []
    model.train()
    for batch in train_dataloader:
        optimizer.zero_grad()
        X_batch, _ = batch
        X_batch = X_batch.to("cuda")

        y_hat_batch = model(X_batch)

        loss = loss_function(y_hat_batch, X_batch) # Compute loss

        loss.backward()   # Calculate gradients
        optimizer.step()   # Update weights using defined optimizer

        all_losses.append(loss.item())
    train_loss = np.mean(all_losses)

    losses = []

    with torch.no_grad():
        model.eval()
        for batch in val_dataloader:
            X_batch, _ = batch
            X_batch = X_batch.to("cuda")

            y_hat_batch = model(X_batch)

            loss = loss_function(y_hat_batch, X_batch)

            losses.append(loss.item())

    print(i, '\t', train_loss, np.mean(losses))

## 1.2 Going variational

Putting an explanation based on rigorous mathematical theory aside, variational autoencoders can also be viewed as normal autoencoders, but with regularization on the bottleneck space.
This bottleneck regularization pushes the samples in the latent space to be normally distributed.
In turn, this means that - after training - if we send randomly generated Gaussian noise through the model, the model will generate samples that are similar to the training samples.
In order to get a better understanding of this process: [this blog](https://towardsdatascience.com/understanding-variational-autoencoders-vaes-f70510919f73) provides a really good overview.

An overview image:

<img src=https://learnopencv.com/wp-content/uploads/2020/11/vae-diagram-1-1024x563.jpg width=400>

In order to go variational: we need to adapt three things.
- Instead of generating a single vectore, we'll need to generate two vectors, parametrizing the mean and variance of the estimated distribution of that sample.
- After encoding and before decoding, in the bottleneck space, we need to stochastically sample from that distribution.
- We need to add a loss term that pushes bottleneck points towards a gaussian distribution.

Typically, we let our model generate the mean and the *log*-variance of the model. This ensures that the model can not predict negative variances (which makes no sense).

The following code illustrates step 1: how to extend the encoder with two layers so that two outputs are obtained:

In [ ]:
x = torch.randn(8, 1, 28, 28)
print(x.shape)
encoder = Encoder(bottleneck_size = 8)
z = encoder(x)
print(z.shape)

# two separate linear combinations separate the bottleneck space into means and logvars:
linear_to_means = nn.Linear(8, 8)
linear_to_logvars = nn.Linear(8, 8)

z_means = linear_to_means(z)
z_logvars = linear_to_logvars(z)
print(z_means.shape)
print(z_logvars.shape)

The following code shows step 2: how to resample from a mean and a log-variance, as how it is done in the [original VAE paper](https://arxiv.org/abs/1312.6114):

In [ ]:
def reparameterization(mean, logvar):
    std = torch.exp(0.5 * logvar)
    eps = torch.randn_like(std)
    return mean + eps * std

means = torch.randn(8, 8)
logvars = torch.randn(8, 8)

y = reparameterization(means, logvars)
print(y.shape)

The following code shows an adaptation of the loss function (step 3).
It shows that the loss still contains the reconstruction error (as with normal autoencoders), but an extra term is added: the [KL divergence](https://en.wikipedia.org/wiki/Kullback%E2%80%93Leibler_divergence). The KL divergence measures how much the estimated means and logvars of the model deviate from a normal distribution. So, by incorporating this into our loss, the model will be stimulated to push these values to a Gaussian distribution. Beta is an additional parameter that weighs the loss component (higher values of beta means that the loss will place higher weight on optimizing the KL divergence instead of focusing on reconstruction).


In [ ]:
import torch.nn.functional as F

class VAELoss(nn.Module):
    def __init__(self, beta = 0.001):
        super().__init__()
        self.beta = beta

    def forward(self, x_reconstruct, means, logvars, x_true):
        reconstruction_loss = F.mse_loss(x_reconstruct, x_true)
        KL_div_loss = (-0.5 * torch.sum(1 + logvars - means**2 - logvars.exp())) / means.size(1)

        return reconstruction_loss + self.beta * KL_div_loss

<div class="alert alert-success">
<b>EXERCISE:</b>
<p> Use the examples of the steps below to reimplemt the autoencoder to make it variational. You can use the code below as a guideline. It should not require more than some copy pasting in the right places.</p>
</div>

In [ ]:
class VariationalAutoEncoder(nn.Module):
    def __init__(self, bottleneck_size = 8):
        super().__init__()
        self.encoder = Encoder(bottleneck_size = bottleneck_size)

        # YOUR CODE HERE, define two separate linear projections.
        self.linear_to_means = ...
        self.linear_to_logvars = ...

        self.decoder = Decoder(bottleneck_size = bottleneck_size)

    def encode(self, x):
        z = self.encoder(x)
        # YOUR CODE HERE, use the two separate linear projections.
        z_means = ...
        z_logvars = ...
        return z_means, z_logvars
    def decode(self, x):
        return self.decoder(x)
    def forward(self, x):
        z_means, z_logvars = self.encode(x)
        # YOUR CODE HERE, sample from z_means and z_logvars using the reparameterization trick
        z = ...
        x_reconstruct = self.decode(z)
        # Note that we return three vectors with this model! This is because we need these three to compute the loss
        return x_reconstruct, z_means, z_logvars

    def reparameterization(self, mean, logvar):
        ...

model = VariationalAutoEncoder(bottleneck_size = 8)

A couple of things will need to be changed compared to the previous training code:
- We use our own custom VAELoss function
- Our model returns multiple outputs, which are all used in the loss function

In [ ]:
N_EPOCHS = 20

model = VariationalAutoEncoder(bottleneck_size = 8).to("cuda")

loss_function = VAELoss(beta = 0.001)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

for i in range(1, N_EPOCHS + 1):
    all_losses = []
    model.train()
    for batch in train_dataloader:
        optimizer.zero_grad()
        X_batch, _ = batch
        X_batch = X_batch.to("cuda")

        x_reconstruct, z_means, z_logvars = model(X_batch)
        loss = loss_function(x_reconstruct, means, logvars, X_batch) # Compute loss

        loss.backward()   # Calculate gradients
        optimizer.step()   # Update weights using defined optimizer

        all_losses.append(loss.item())
    train_loss = np.mean(all_losses)

    losses = []

    with torch.no_grad():
        model.eval()
        for batch in val_dataloader:
            X_batch, _ = batch
            X_batch = X_batch.to("cuda")

            x_reconstruct, z_means, z_logvars = model(X_batch)
            loss = loss_function(x_reconstruct, means, logvars, X_batch) # Compute loss

            losses.append(loss.item())

    print(i, '\t', train_loss, np.mean(losses))

## 1.3 Dimensionality reduction and sampling

Now that we have trained our model, let's see how good it is at dimensionality reduction. For this purpose, let's repeat the validation loop, but this time only use the encoder of the model:

In [ ]:
encoded_inputs = []
classes = []
with torch.no_grad():
    model.eval()
    for batch in val_dataloader:
        X_batch, y_batch = batch
        X_batch = X_batch.to("cuda")

        z = model.linear_to_means(model.encoder(X_batch))
        encoded_inputs.append(z)
        classes.append(y_batch)

classes = torch.cat(classes).cpu().numpy()
encoded_inputs = torch.cat(encoded_inputs).cpu().numpy()

print(encoded_inputs.shape)

Let's use t-SNE -- a dimensionality reduction technique -- to reduce our encodings to two-dimensional vectors, which can be visualized. 

In [ ]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")
sns.set_palette("pastel")

encoded_tsne = TSNE(verbose = 10, init = "pca", learning_rate = "auto").fit_transform(encoded_inputs)
plt.figure(figsize = (5, 5))
sns.scatterplot(x = encoded_tsne[:,0], y = encoded_tsne[:, 1], hue = classes, alpha = .75, palette="pastel", s = 3, linewidth = 0)
plt.show()

We can see that the model embeds samples of the class together, in clusters.

Next, let's try to generate some new samples from our learned distribution. 

Remember that our variational autoencoder has been trained to represent samples as Gaussians in the bottleneck space. Logically, this allows to sample random Gaussians and retrieve realistic-looking samples after decoding. 

In [ ]:
# generating some random noise
samples = torch.randn(16, 8).to("cuda")
with torch.no_grad():
    model.eval()
    decoded_samples = model.decode(samples)
    decoded_samples = torch.clip(decoded_samples, min = 0, max = 1).cpu().numpy() * 255

figure = plt.figure(figsize=(10, 8))
cols, rows = 4, 4
for i in range(cols * rows):
    img = decoded_samples[i]
    figure.add_subplot(rows, cols, i+1)
    plt.axis("off")
    plt.imshow(img.reshape(-1, 28, 28).squeeze(), cmap="gray")
plt.show()